In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
%matplotlib inline

In [2]:
from dashboards_classes import teach_main

In [7]:
from IPython.display import display, HTML

In [8]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 180px
}
</style>

In [10]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
#@title **Demo Application**

import numpy as np
import ipywidgets as widgets

np.seterr(divide='ignore', invalid='ignore')
    
# Tool outputs are created and the Teach Main class is instantiated

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab([out1,out2,out3,out4], layout=widgets.Layout(width="1300px"))

tab.set_title(0, 'Upload')
tab.set_title(1, 'View Trajectory Data')
tab.set_title(2, 'Intrinsic Evaluation')
tab.set_title(3, 'Extrinsic Evaluation')

display(tab)

TM = teach_main.Teach_Main(out3)

with out1:
    
    display(TM.upload_accordion)

with out2:

    display(TM.VTDO.view_trajectory_data_box)

with out3:

    display(TM.IEO.intrinsic_evaluation_box)

with out4:

    display(TM.EEO.extrinsic_evaluation_box)
